<a href="https://colab.research.google.com/github/mohyunyang/MyStudy/blob/master/DL_NLP(Text_Preprocessing2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인코딩
- 컴퓨터는 텍스트보다는 숫자를 더 잘 처리 할 수 있습니다.
- 이를 위해 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있습니다. 그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있습니다.

- 예를 들어 갖고 있는 텍스트에 단어가 5,000개가 있다면, 5,000개의 단어들 각각에 1번부터 5,000번까지 단어와 맵핑되는 고유한 정수. 다른 표현으로는 인덱스를 부여합니다. 
- 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 숫자가 부여됩니다. 
- ★ 인덱스를 부여하는 방법은 여러 가지가 있을 수 있는데 랜덤으로 부여하기도 하지만, 보통은 단어 등장 빈도수를 기준으로 정렬한 뒤에 부여합니다.

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [ ]:
# 문장토큰화
sentences = sent_tokenize(raw_text)
print(sentences)

1. 먼저 문장들이 있다. 문장들 중에서 한 문장씩 뽑는다. 뽑아서 한 단어씩 토큰화를 시킨다. 
2. 토큰화시킨 문장 중 불용어를 거른다.
3. 의미있는 토큰만 남았을 터, 이들만의 리스트를 만든다.
4. 동시에 빈딕셔너리를 통해 단어가 몇번 나왔는지 카운트한다.

In [ ]:
vocab = {}
preprocessed_sentences = []  # 사전처리된 문장들
stop_words = set(stopwords.words('english')) # 불용어 집합설정

for sentence in sentences: # sentence 는 "A barber is a person. 
    # 단어 토큰화
    tokenized_sentence = word_tokenize(sentence) # ['A','barber', 'is', 'a', 'person']
    result = [] # 불용어가 아닌 토큰만 모인다

    for word in tokenized_sentence:  # word = ['A'] ['barber'] ['is'] ['person'] 
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다.
        if word not in stop_words: #  불용어집합안에 word가 없다면
            if len(word) > 2: # 그리고 word의 길이가 3이상이라면
                result.append(word) # result = [] 에 word 추가 
                if word not in vocab: # vocab = {} 안에 word가 없다면 
                  vocab[word] = 1  # 'word' 로 키를 만들고 벨류로 0 설정//
                else:
                  vocab[word] += 1 # 1을 추가
             
    preprocessed_sentences.append(result) 
print(preprocessed_sentences)

In [ ]:
# 빈도수 확인
print(vocab)

In [ ]:
# 'barber'라는 단어의 빈도수 출력
print(vocab["barber"])

In [ ]:
# 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True) # 딕셔너리로는 정렬안되니 items 를 써서 튜플형태로, key는 파라미터이름 ('barber', 8) x[1]= 8
print(vocab_sorted)

In [ ]:
# 높은 빈도수를 가진 단어일수록 낮은 정수를 부여. 정수는 1부터 부여.

word_to_index = {} # 딕셔너리 생성
i = 0
for (word, frequency) in vocab_sorted :   # ('barber', 8) , word = 'barber' , frequency = 8
    if frequency > 1 : # 빈도수가 작은 단어는 제외.
        i = i + 1
        word_to_index[word] = i

print(word_to_index)

In [ ]:
vocab_size = 5  # 인덱스가 5 라는 뜻

# 인덱스가 5 초과인 단어 제거
words_frequency = [word for word, index in word_to_index.items() if index > vocab_size] # ['word', 'keeping']
print(words_frequency)

# 해당 단어에 대한 인덱스 정보를 삭제
for w in words_frequency:
  del word_to_index[w] # {word': 6, 'keeping': 7}
print(word_to_index)

# 지쳐서 할 수가 없기 때문에 작은 것 부터 정리를 빨리 해가는 걸 추천

In [ ]:
word_to_index['OOV'] = len(word_to_index) + 1 # Out Of Vocabulary # len(word_to_index) = 6
print(word_to_index)

In [ ]:
# 단어 집합에 있는 단어라면 해당 단어의 숫자가 없으면 OOV 의 숫자를 반환
encoded_sentences = [] # 1
for sentence in preprocessed_sentences: # [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person']...
  encoded_sentence = [] # 2  [1,5]
  for word in sentence:  # 'barber'  ['barber', 'person']
    # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
    try:
      encoded_sentence.append(word_to_index[word]) # 2번에 추가 (1)
    # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
    except KeyError: # 'good' 은 없기 때문에 없으면 keyError 뜸
      encoded_sentence.append(word_to_index['OOV'])
  encoded_sentences.append(encoded_sentence) # [[]] 처리
print(encoded_sentences)

지금까지 파이썬의 dictionary 자료형으로 정수 인코딩을 진행해보았습니다. 그런데 이보다는 좀 더 쉽게 하기 위해서 Counter, FreqDist, enumerate를 사용하거나, 케라스 토크나이저를 사용하는 것을 권장합니다.

## Counter

In [ ]:
from collections import Counter
print(preprocessed_sentences)

all_word_list = sum(preprocessed_sentences,[]) # 리스트끼리 + 안됨, []를 써서 벗기기
print(all_word_list)

In [ ]:
vocab = Counter(all_word_list) # 딕셔너리 형태로 만듦
print(vocab)
print(vocab['barber'])

In [ ]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 가장빈도수가 높은 것 부터5번째까지 끊겠다.
vocab # 리스트안에 튜플형태

In [ ]:
word_to_index = {}
i = 0
for (word, frequency) in vocab : # ('barber', 8) // word = barber, frequency = 8
    i = i + 1
    word_to_index[word] = i

print(word_to_index)

## NLTK의 FreqDist 사용하기

- NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원합니다. 위에서 사용한 Counter()랑 같은 방법으로 사용할 수 있습니다.



In [ ]:
from nltk import FreqDist
import numpy as np

vocab = FreqDist(np.hstack(preprocessed_sentences))
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

word_to_index = {word[0]:index + 1 for index, word in enumerate(vocab)} # enumerate() : 인덱스와 원소로 이루어진 터플(tuple)을 만들어줍니다.
print(word_to_index)

In [ ]:
for index, word in enumerate(vocab):
  print(index, word)

## 케라스(Keras)의 텍스트 전처리
- 케라스(Keras)는 기본적인 전처리를 위한 도구들을 제공합니다. 때로는 정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저를 사용하기도 하는데, 사용 방법과 그 특징에 대해서 이해해보겠습니다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성
print(tokenizer.fit_on_texts(preprocessed_sentences))

# 각 단어에 인덱스가 어떻게 부여되었는지를 보려면, word_index를 사용
print(tokenizer.word_index)

# 각 단어가 카운트를 수행하였을 때 몇 개였는지를 보고자 한다면 word_counts를 사용
print(tokenizer.word_counts)

# texts_to_sequences()는 입력으로 들어온 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환
print(tokenizer.texts_to_sequences(preprocessed_sentences))

In [ ]:
vocab_size = 5

# 케라스 토크나이저에서는 tokenizer = Tokenizer(num_words=숫자)와 같은 방법으로
# 빈도수가 높은 상위 몇 개의 단어만 사용하겠다고 지정할 수 있습니다.
tokenizer = Tokenizer(num_words = vocab_size + 1) # 상위 5개 단어만 사용 # num_words에서 +1을 더해서 값을 넣어주는 이유는 num_words는 숫자를 0부터 카운트합니다. 패딩(padding)이라는 작업 뒤에 등장
tokenizer.fit_on_texts(preprocessed_sentences) # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성

print(tokenizer.word_index)

In [ ]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

In [ ]:
vocab_size = 5
words_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size + 1] 

# 인덱스가 5 초과인 단어 제거
for word in words_frequency:
    del tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

In [ ]:
# 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

In [ ]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

In [ ]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

## 원-핫 인코딩(One-Hot Encoding)[링크 텍스트](https:// [링크 텍스트](https://))

- 컴퓨터 또는 기계는 문자보다는 숫자를 더 잘 처리 할 수 있습니다. 이를 위해 자연어 처리에서는 문자를 숫자로 바꾸는 여러가지 기법들이 있습니다. 원-핫 인코딩(One-Hot Encoding)은 그 많은 기법 중에서 단어를 표현하는 가장 기본적인 표현 방법이며, 머신 러닝, 딥 러닝을 하기 위해서는 반드시 배워야 하는 표현 방법입니다.

In [ ]:
from konlpy.tag import Okt  

okt = Okt()  
tokens = okt.morphs("나는 자연어 처리를 배운다")  
print(tokens)

In [ ]:
# 각 토큰에 대해서 고유한 정수를 부여합니다. 지금은 문장이 짧기 때문에 각 단어의 빈도수를 고려하지 않지만, 빈도수 순으로 단어를 정렬하여 정수를 부여하는 경우가 많습니다.

word_to_index = {word : index for index, word in enumerate(tokens)}
print('단어 집합 :',word_to_index)

In [ ]:
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*(len(word_to_index)) # [0]*6   # 데이터 개수만큼 차원을 늘린다. one_hot_vector = [0, 0, 0, 0, 0, 0] 
  index = word_to_index[word]  # word_to_index['자연어'] = 2 , index = 2
  one_hot_vector[index] = 1 # one_hot_vector[2] = 1
  return one_hot_vector

print(one_hot_encoding("나", word_to_index))
print(one_hot_encoding("는", word_to_index))
print(one_hot_encoding("자연어", word_to_index))

## 케라스(Keras)를 이용한 원-핫 인코딩(One-Hot Encoding)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
text = " 나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야 "
tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성
tokenizer.fit_on_texts([text])  #  {'갈래': 3, '점심': 2 ..} 이렇게 세어준다.
print('단어집합:', tokenizer.word_index)

sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"

# texts_to_sequences()는 입력으로 들어온 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환
encoded = tokenizer.texts_to_sequences([sub_text])[0] # tokenizer.word_index 작업 후에 가능
print(encoded)

one_hot = to_categorical(encoded)
print(one_hot)